In [ ]:
import json
import requests
from census import Census
from us import states
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gmaps
import gmaps.geojson_geometries
from matplotlib.cm import viridis
from matplotlib.colors import to_hex
import os
import gmaps.datasets
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
gkey = os.environ.get('gmaps')
gmaps.configure(api_key=gkey)
plotly.tools.set_credentials_file(username='willdoucet', api_key='PsaRlgAQKeEaFhQe1b6B')
c = Census("cb3da972a3410fea907ff2284991388d72455cad")

In [ ]:
import plotly.graph_objs as go
import plotly.io as pio

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#assign ranks to rows GENERAL
ranks=[]
basescores=[]
i=1
for i in range (1, 59, 1):
    ranks.append(i)
basescores=list(reversed(ranks))
print(basescores)

In [ ]:
state_fips = '06'
fips_list_with_state = []
fips_df = pd.read_csv('data/fips.csv', dtype=str)
fips_df

fips_list = fips_df['FIPS'].tolist()
county_names = fips_df['Name'].tolist()

for value in fips_list:
    
    fips_list_with_state.append(f'06{value}')
    
fips_list_with_state

In [ ]:
count = 0
for f in fips_list:
    
    #grab data
    data = c.acs5.state_county(('NAME','B25013_003E', 'B25013_004E','B25013_005E','B25013_006E','B25013_008E','B25013_009E','B25013_010E',
                      'B25013_011E'), state_fips, f, year=2016)
        
    temp_df = pd.DataFrame(data)
    #when count = 0 create initial housing_df dataframe
    if count == 0:
            
        housing_df = pd.DataFrame(temp_df)
    #when count > 0 append new data to housing_df dataframe    
    if count > 0:
            
        housing_df = housing_df.append(temp_df)
            
    count +=1

In [ ]:
housing_df = housing_df.rename(columns={'B25013_003E': 'Owner - No High School', 'B25013_004E': 'Owner - High School',
                                        'B25013_005E': 'Owner - Some College or Associates Deg', 'B25013_006E': 'Owner - Bachelors',
                                       'B25013_008E': 'Renter - No High School', 'B25013_009E': 'Renter - High School',
                                        'B25013_010E': 'Renter - Some College or Associates Deg', 'B25013_011E': 'Renter - Bachelors',
                                        'NAME': 'County Name', 'county': 'County Code'
                                       })

In [ ]:
del housing_df['state']

In [ ]:
housing_df.set_index(['County Code', 'County Name'], inplace=True)

In [ ]:
housing_df

In [ ]:
count = 0
for f in fips_list:
    
    #grab data
    data = c.acs5.state_county(('NAME', 'B08303_002E', 'B08303_003E', 'B08303_004E', 'B08303_005E', 'B08303_006E',
                             'B08303_007E', 'B08303_008E', 'B08303_009E', 'B08303_010E',
                             'B08303_011E', 'B08303_012E', 'B08303_013E'), state_fips, f, year=2016)
        
    temp_df = pd.DataFrame(data)
    #when count = 0 create initial commute_df dataframe
    if count == 0:
            
        commute_df = pd.DataFrame(temp_df)
    #when count > 0 append new data to commute_df dataframe    
    if count > 0:
            
        commute_df = commute_df.append(temp_df)
            
    count +=1

In [ ]:
commute_df.rename(columns={'B08303_002E': '<5', 'B08303_003E': '5-9', 'B08303_004E': '10-14', 'B08303_005E': '15-19',
                           'B08303_006E': '20-24','B08303_007E': '25-29', 'B08303_008E': '30-34', 'B08303_009E': '35-39',
                           'B08303_010E': '40-44', 'B08303_011E': '45-59', 'B08303_012E': '60-89', 'B08303_013E': '>90',
                          'NAME': 'County Name', 'county': 'County Code'
                          }, inplace=True)

In [ ]:
del commute_df['state']

commute_df.set_index(['County Code', 'County Name'], inplace=True)

for colname in commute_df.columns:
    commute_df[colname] = commute_df[colname].astype(float)

In [ ]:
commute_df.to_csv('data/Commute Time By County.csv')

In [ ]:
commute_med_num = []
commute_l = commute_df.sum(axis=1).tolist()

x= 0
commute_bracket = []
commute_tier = []
for value in commute_l:
    
    commute_med_num.append(value / 2)
    
commute_med_num

for index, row in commute_df.iterrows():
    count=0
    y=0
    for colname in commute_df.columns:
        
        if count <= commute_med_num[x]:
            
            count += row[colname]
            y+=1
            
        elif count > commute_med_num[x]:
            
            commute_bracket.append(colname)
            commute_tier.append(y)
            
            break
    x += 1

In [ ]:
commute_colors = []
commute_tier_norm = []
for value in commute_tier:
    
    commute_denom = len(commute_df.columns)
    normalized_commute = value / commute_denom
    
    commute_tier_norm.append(normalized_commute)

In [ ]:
commute_colorscale = [
    '#c2d1f0',
    '#99b3e6',
    '#7094db',
    '#4775d1',
    '#2e5cb8',
    '#193366'
]

commute_choro = ff.create_choropleth(fips=fips_list_with_state, values=commute_bracket, colorscale=commute_colorscale, scope=['CA'],
                                    county_outline={'color': 'rgb(255,255,255)', 'width': .5}, legend_title = 'Time in Minutes',
                                    title = 'Median Commute Time by County' 
                                    )
py.iplot(commute_choro, filename='Commute Time by County')

In [ ]:
pio.write_image(commute_choro, 'images/commute_choro.png')

In [ ]:
count = 0
for f in fips_list:
    
    #grab data
    data = c.acs5.state_county(('NAME', 'B19001_002E', 'B19001_003E', 'B19001_004E', 'B19001_005E', 'B19001_006E',
                         'B19001_007E', 'B19001_008E', 'B19001_009E', 'B19001_010E', 'B19001_011E',
                         'B19001_012E', 'B19001_013E', 'B19001_014E', 'B19001_015E', 'B19001_016E',
                         'B19001_017E'), state_fips, f, year=2016)
        
    temp_df = pd.DataFrame(data)
    #when count = 0 create initial income_df dataframe
    if count == 0:
            
        income_df = pd.DataFrame(temp_df)
    #when count > 0 append new data to income_df dataframe    
    if count > 0:
            
        income_df = income_df.append(temp_df)
            
    count +=1

In [ ]:
income_df.rename(columns={'B19001_002E': '<.100', 'B19001_003E': '.10,000-.14,999', 'B19001_004E': '.15,000-.19,999',
                          'B19001_005E': '.200-.249', 'B19001_006E': '.250-.299', 'B19001_007E': '.300-.349',
                          'B19001_008E': '.350-.399', 'B19001_009E': '.400-.449', 'B19001_010E': '.450-.499',
                          'B19001_011E': '.500-.599', 'B19001_012E': '.600-.749', 'B19001_013E': '.750-.999',
                          'B19001_014E': '1.000-1.249', 'B19001_015E': '1.250-1.499', 'B19001_016E': '1.500-1.999',
                         'B19001_017E': '>2.000',
                          'NAME': 'County Name', 'county': 'County Code'
                          }, inplace=True)

In [ ]:
del income_df['state']

income_df.set_index(['County Code', 'County Name'], inplace=True)

for colname in income_df.columns:
    income_df[colname] = income_df[colname].astype(float)

In [ ]:
income_df

In [ ]:
income_med_num = []
income_l = income_df.sum(axis=1).tolist()

x= 0
income_bracket = []
income_tier = []
for value in income_l:
    
    income_med_num.append(value / 2)
    
income_med_num

for index, row in income_df.iterrows():
    count=0
    y=0
    for colname in income_df.columns:
        
        if count <= income_med_num[x]:
            
            count += row[colname]
            y+=1
            
        elif count > income_med_num[x]:
            
            income_bracket.append(colname)
            income_tier.append(y)
            
            break
    x += 1


In [ ]:
income_df.to_csv('data/Income by County.csv')

In [ ]:
income_colorscale = ['#d9f2d9', '#b3e6b3', '#8cd98c', '#66cc66', '#39ac39', '#2d862d', '#133913']

income_choro = ff.create_choropleth(fips=fips_list_with_state, values=income_bracket, colorscale=income_colorscale, scope=['CA'],
                                   county_outline={'color': 'rgb(255,255,255)', 'width': .5}, legend_title = 'Median Income / 100,000',
                                    title = 'Median Income by County'
                                   )
py.iplot(income_choro, filename='Household Income by County')

In [ ]:
pio.write_image(income_choro, 'images/Income Choro.png')

In [ ]:
#Percent Rich is Here
income_rank_df = income_df

income_rank_df['Total People'] = income_l

income_rank_df['Percentage Rich'] = income_rank_df['>2.000'] / income_rank_df['Total People'] * 100

income_rank_df.sort_values('Percentage Rich', ascending=False)

In [ ]:
income_ranking_df=income_rank_df.sort_values('Percentage Rich', ascending=False)
#RANKS INCOME
income_ranking_df['Rank']=ranks
income_ranking_df.rename(columns={'Rank': 'Wealth Rank'}, inplace=True)
income_ranking_df

In [ ]:
count = 0
for f in fips_list:
    
    #grab data
    data = c.acs5.state_county(('NAME', 'B15003_017E', 'B15003_018E', 'B15003_019E', 'B15003_020E', 'B15003_021E',
                         'B15003_022E', 'B15003_023E', 'B15003_024E', 'B15003_025E'), state_fips, f, year=2016)
        
    temp_df = pd.DataFrame(data)
    #when count = 0 create initial education_df dataframe
    if count == 0:
            
        education_df = pd.DataFrame(temp_df)
    #when count > 0 append new data to education_df dataframe    
    if count > 0:
            
        education_df = education_df.append(temp_df)
            
    count +=1

In [ ]:
education_df.rename(columns={'B15003_017E': 'High School', 'B15003_018E': 'GED or Equivalent', 'B15003_019E': '<1yr College',
                             'B15003_020E': '>=1yr College No Deg', 'B15003_021E': 'Associates Deg',
                         'B15003_022E': 'Bachelors Deg', 'B15003_023E': 'Masters Deg', 'B15003_024E': 'Professional Deg',
                         'B15003_025E': 'PhD',
                          'NAME': 'County Name', 'county': 'County Code'
                          }, inplace=True)

In [ ]:
del education_df['state']

education_df.set_index(['County Code', 'County Name'], inplace=True)

for colname in education_df.columns:
    education_df[colname] = education_df[colname].astype(float)

In [ ]:
education_med_num = []
education_l = education_df.sum(axis=1).tolist()

x= 0
education_bracket = []
education_tier = []
for value in education_l:
    
    education_med_num.append(value / 2)
    
education_med_num

for index, row in education_df.iterrows():
    count=0
    y=0
    for colname in education_df.columns:
        
        if count <= education_med_num[x]:
            
            count += row[colname]
            y+=1
            
        elif count > education_med_num[x]:
            
            education_bracket.append(colname)
            education_tier.append(y)
            
            break
    x += 1

In [ ]:
education_df.to_csv('data/Education DataFrame')

In [ ]:
education_colorscale = ["#ebf3fb","#b3d2e9",
              "#57a0ce","#1361a9",
              "#08306b"]

education_choro = ff.create_choropleth(fips=fips_list_with_state, values=education_bracket, colorscale=education_colorscale, scope=['CA'],
                                      county_outline={'color': 'rgb(255,255,255)', 'width': .5}, legend_title = 'Education Level',
                                    title = 'Education Level by County'
                                      )
py.iplot(education_choro, filename='Level of Education by County')

In [ ]:
pio.write_image(education_choro, 'images/Education Choro.png')

In [ ]:
education_rank_df = education_df

education_rank_df['Educated'] = education_rank_df['Bachelors Deg'] + education_rank_df['Masters Deg'] + education_rank_df['Professional Deg'] + education_rank_df['PhD']

In [ ]:
education_rank_df['Percent Educated'] = education_l

education_rank_df.rename(columns={'Percent Educated': 'Total People'}, inplace=True)

education_rank_df

In [ ]:
education_rank_df['Percent Educated'] = education_rank_df['Educated'] / education_rank_df['Total People'] * 100
education_rank_df.sort_values('Percent Educated', ascending=False)

In [ ]:
#Percent educated is here and so are ranks
education_rank_df['Percent Educated'] = education_rank_df['Educated'] / education_rank_df['Total People'] * 100
#education_rank_df.sort_values('Percent Educated', ascending=False)
#education_rank_df
#RANKS
education_ranking_df=education_rank_df.sort_values('Percent Educated', ascending=False)
#education_ranking_df
education_ranking_df['Rank']=ranks
education_ranking_df.rename(columns={'Rank': 'Education Rank'}, inplace=True)
education_ranking_df

In [ ]:
count = 0
for f in fips_list:
    
    #grab data
    data = c.acs5.state_county(('NAME', 'B25087_004E', 'B25087_005E', 'B25087_006E', 'B25087_007E', 'B25087_008E',
                         'B25087_009E', 'B25087_010E', 'B25087_011E', 'B25087_012E', 'B25087_013E', 'B25087_014E',
                         'B25087_015E', 'B25087_016E', 'B25087_017E'), state_fips, f, year=2016)
        
    temp_df = pd.DataFrame(data)
    #when count = 0 create initial mortgage_df dataframe
    if count == 0:
            
        mortgage_df = pd.DataFrame(temp_df)
    #when count > 0 append new data to mortgage_df dataframe    
    if count > 0:
            
        mortgage_df = mortgage_df.append(temp_df)
            
    count +=1

In [ ]:
mortgage_df.rename(columns={'B25087_004E': '.200-.299', 'B25087_005E': '.300-.399', 'B25087_006E': '.400-.499',
                            'B25087_007E': '.500-.599', 'B25087_008E': '.600-.699', 'B25087_009E': '.700-.799',
                            'B25087_010E': '.800-.899', 'B25087_011E': '.900-.999', 'B25087_012E': '1.000-1.249',
                            'B25087_013E': '1.250-1.499', 'B25087_014E': '1.500-1.999', 'B25087_015E': '2.000-2.499',
                            'B25087_016E': '2.500-2.999', 'B25087_017E': '>3.000',
                          'NAME': 'County Name', 'county': 'County Code'
                          }, inplace=True)

In [ ]:
del mortgage_df['state']

mortgage_df.set_index(['County Code', 'County Name'], inplace=True)

for colname in mortgage_df.columns:
    mortgage_df[colname] = mortgage_df[colname].astype(float)

In [ ]:
mortgage_med_num = []
mortgage_l = mortgage_df.sum(axis=1).tolist()

x= 0
mortgage_bracket = []
mortgage_tier = []
for value in mortgage_l:
    
    mortgage_med_num.append(value / 2)
    
mortgage_med_num

for index, row in mortgage_df.iterrows():
    count=0
    y=0
    for colname in mortgage_df.columns:
        
        if count <= mortgage_med_num[x]:
            
            count += row[colname]
            y+=1
            
        elif count > mortgage_med_num[x]:
            
            mortgage_bracket.append(colname)
            mortgage_tier.append(y)
            
            break
    x += 1

In [ ]:
mortgage_df.to_csv('data/Mortage DataFrame')

In [ ]:
mortgage_colorscale = ["#ebf3fb","#d2e3f3","#b3d2e9",
              "#85bcdb","#57a0ce","#3082be","#1361a9",
              "#0b4083","#08306b"]

mortgage_choro = ff.create_choropleth(fips=fips_list_with_state, values=mortgage_bracket, colorscale=mortgage_colorscale, scope=['CA'],
                                     county_outline={'color': 'rgb(255,255,255)', 'width': .5}, legend_title = 'Mortgage Amount / 1000',
                                    title = 'Mortgage by County'
                                     )
py.iplot(mortgage_choro, filename='Monthly Mortage Payment by County')

In [ ]:
pio.write_image(mortgage_choro, 'images/Mortgage Choro.png')

In [ ]:
mortgage_df

In [ ]:
mortgage_rank_df = mortgage_df

mortgage_rank_df

In [ ]:
count = 0
for f in fips_list:
    
    #grab data
    data = c.acs5.state_county(('NAME', 'B25091_002E', 'B25091_003E', 'B25091_004E', 'B25091_005E', 'B25091_006E',
                         'B25091_007E', 'B25091_008E', 'B25091_009E', 'B25091_010E', 'B25091_011E'), state_fips, f, year=2016)
        
    temp_df = pd.DataFrame(data)
    #when count = 0 create initial mortgage_per_df dataframe
    if count == 0:
            
        mortgage_per_df = pd.DataFrame(temp_df)
    #when count > 0 append new data to mortgage_per_df dataframe    
    if count > 0:
            
        mortgage_per_df = mortgage_per_df.append(temp_df)
            
    count +=1

In [ ]:
mortgage_per_df.rename(columns={'B25091_002E': 'Total', 'B25091_003E': '<10%', 'B25091_004E': '10%-14%',
                                                 'B25091_005E': '15%-19%', 'B25091_006E': '20%-24%', 'B25091_007E': '25%-29%',
                                                 'B25091_008E': '30%-34%', 'B25091_009E': '35%-39%', 'B25091_010E': '40%-49%',
                                                 'B25091_011E': '>50%',
                          'NAME': 'County Name', 'county': 'County Code'
                          }, inplace=True)

In [ ]:
del mortgage_per_df['state']

mortgage_per_df.set_index(['County Code', 'County Name'], inplace=True)

for colname in mortgage_per_df.columns:
    mortgage_per_df[colname] = mortgage_per_df[colname].astype(float)

In [ ]:
del mortgage_per_df['Total']

In [ ]:
mortgage_per_med_num = []
mortgage_per_l = mortgage_per_df.sum(axis=1).tolist()

x= 0
mortgage_per_bracket = []
mortgage_per_tier = []
for value in mortgage_per_l:
    
    mortgage_per_med_num.append(value / 2)
    
mortgage_per_med_num

for index, row in mortgage_per_df.iterrows():
    count=0
    y=0
    for colname in mortgage_per_df.columns:
        
        if count <= mortgage_per_med_num[x]:
            
            count += row[colname]
            y+=1
            
        elif count > mortgage_per_med_num[x]:
            
            mortgage_per_bracket.append(colname)
            mortgage_per_tier.append(y)
            
            break
    x += 1

In [ ]:
mortgage_per_bracket

In [ ]:
mortgage_per_df.to_csv('data/Mortage as Percentage of Income DataFrame.csv')

In [ ]:
mortgage_per_colorscale = ["#b3d2e9",
              "#85bcdb","#3082be",
              "#0b4083","#08306b"]

mortgage_per_choro = ff.create_choropleth(fips=fips_list_with_state, values=mortgage_per_bracket, colorscale=mortgage_per_colorscale, scope=['CA'],
                                         county_outline={'color': 'rgb(255,255,255)', 'width': .5}, legend_title = 'Mortgage/Income',
                                    title = 'Mortgage as Percent of Income'
                                         )
py.iplot(mortgage_per_choro, filename='Percentage of Income Devoted to Mortgage')

In [ ]:
count = 0
for f in fips_list:
    
    #grab data
    data = c.acs5.state_county(('NAME', 'B25070_001E', 'B25070_002E', 'B25070_003E', 'B25070_004E', 'B25070_005E',
                         'B25070_006E', 'B25070_007E', 'B25070_008E', 'B25070_009E', 'B25070_010E'), state_fips, f, year=2016)
        
    temp_df = pd.DataFrame(data)
    #when count = 0 create initial rent_per_df dataframe
    if count == 0:
            
        rent_per_df = pd.DataFrame(temp_df)
    #when count > 0 append new data to rent_per_df dataframe    
    if count > 0:
            
        rent_per_df = rent_per_df.append(temp_df)
            
    count +=1

In [ ]:
rent_per_df.rename(columns={'B25070_001E': 'Total', 'B25070_002E': '<10', 'B25070_003E': '10-14',
                                                 'B25070_004E': '15-19', 'B25070_005E': '20-24', 'B25070_006E': '25-29',
                                                 'B25070_007E': '30-34', 'B25070_008E': '35-39', 'B25070_009E': '40-49',
                                                 'B25070_010E': '>50',
                          'NAME': 'County Name', 'county': 'County Code'
                          }, inplace=True)

In [ ]:
del rent_per_df['state']
del rent_per_df['Total']
rent_per_df.set_index(['County Code', 'County Name'], inplace=True)

for colname in rent_per_df.columns:
    rent_per_df[colname] = rent_per_df[colname].astype(float)

In [ ]:
rent_per_med_num = []
rent_per_l = rent_per_df.sum(axis=1).tolist()

x= 0
rent_per_bracket = []
rent_per_tier = []
for value in rent_per_l:
    
    rent_per_med_num.append(value / 2)
    
rent_per_med_num

for index, row in rent_per_df.iterrows():
    count=0
    y=0
    for colname in rent_per_df.columns:
        
        if count <= rent_per_med_num[x]:
            
            count += row[colname]
            y+=1
            
        elif count > rent_per_med_num[x]:
            
            rent_per_bracket.append(colname)
            rent_per_tier.append(y)
            
            break
    x += 1

In [ ]:
rent_per_df.to_csv('data/Rent as Percentage of Income DataFrame')

In [ ]:
rent_per_colorscale = ["#ebf3fb","#d2e3f3","#b3d2e9",
              "#85bcdb","#57a0ce","#3082be","#1361a9",
              "#0b4083","#08306b"]

rent_per_choro = ff.create_choropleth(fips=fips_list_with_state, values=rent_per_bracket, colorscale=rent_per_colorscale, scope=['CA'])
py.iplot(rent_per_choro, filename='Percentage of Income Devoted to Rent')

In [ ]:
housing_price_df = pd.read_csv('data/zillow_housing_price.csv')
del housing_price_df['Unnamed: 0']
housing_price_list = housing_price_df['Median Value'].tolist()

In [ ]:
housing_price_endpoints = list(np.mgrid[min(housing_price_list):max(housing_price_list):14j])
#housing_price_fig = ff.create_choropleth()
mortgage_price_colorscale = ["#cccccc","#ebf3fb","#deebf7","#d2e3f3","#c6dbef","#b3d2e9","#9ecae1",
              "#85bcdb","#6baed6","#57a0ce","#4292c6","#3082be","#2171b5","#1361a9",
              "#08519c","#0b4083","#08306b"]
housing_price_endpoints

In [ ]:
housing_price_fig = ff.create_choropleth(fips = fips_list_with_state, values = housing_price_list,
                                        colorscale = mortgage_price_colorscale, binning_endpoints = housing_price_endpoints,
                                         scope = ['CA'],
                                         county_outline={'color': 'rgb(255,255,255)', 'width': .5}, legend_title = 'Median Dollar Value of Homes',
                                    title = 'Median Housing Price by County'

)

In [ ]:
py.iplot(housing_price_fig, filename='Housing Price by County')

In [ ]:
pio.write_image(housing_price_fig, 'images/Housing Value Choro.png')

In [ ]:
income_vs_housing = pd.DataFrame(income_bracket, housing_price_list, index=county_names)

In [ ]:
income_vs_housing = income_vs_housing.reset_index()

In [ ]:
income_vs_housing['County Name'] = county_names

In [ ]:
income_vs_housing.set_index('County Name', inplace=True)

In [ ]:
income_graph_list = income_vs_housing.loc[income_vs_housing['index'] != 0]

In [ ]:
income_graph_list.rename(columns={'index': 'Housing Price', 0: 'Median Income'}, inplace=True)

In [ ]:
#
income_float_list = []
for index, row in income_graph_list.iterrows():
    
    if row['Median Income'] == '.400-.449':
        
        income_float_list.append(425000)
        
    elif row['Median Income'] == '.450-.499':
    
        income_float_list.append(475000)
        
    elif row['Median Income'] == '.500-.549':
        
        income_float_list.append(525000)
    
    elif row['Median Income'] == '.500-.599':
        
        income_float_list.append(550000)
    
    elif row['Median Income'] == '.600-.749':
        
        income_float_list.append(675000)
        
    elif row['Median Income'] == '.650-.699':
        
        income_float_list.append(675000)
        
    elif row['Median Income'] == '.700-.749':
        
        income_float_list.append(725000)
        
    elif row['Median Income'] == '.750-.999':
        
        income_float_list.append(875000)
        
    elif row['Median Income'] == '1.000-1.249':
        
        income_float_list.append(1125000)
        
    elif row['Median Income'] == '1.250-1.499':
        
        income_float_list.append(1375000)
        

In [ ]:
income_graph_list['Average Income'] = income_float_list

In [ ]:
income_graph_list.plot(kind='scatter', x='Average Income', y='Housing Price')
plt.title('Housing Price vs Income')
plt.savefig('images/Housing Price vs Income')

In [ ]:


count = 0
for f in fips_list:
    
    #grab data
    data = c.acs5.state_county(('NAME', 'B25064_001E'), state_fips, f, year=2016)
        
    temp_df = pd.DataFrame(data)
    #when count = 0 create initial rent_df dataframe
    if count == 0:
            
        rent_df = pd.DataFrame(temp_df)
    #when count > 0 append new data to rent_df dataframe    
    if count > 0:
            
        rent_df = rent_df.append(temp_df)
            
    count +=1

In [ ]:
rent_df.rename(columns={'B25070_001E': 'Total', 'B25064_001E': 'Median Rent',
                          'NAME': 'County Name', 'county': 'County Code'
                          }, inplace=True)

In [ ]:
del rent_df['state']
rent_df.set_index(['County Code', 'County Name'], inplace=True)

for colname in rent_df.columns:
    rent_df[colname] = rent_df[colname].astype(float)

In [ ]:
rent_med_num = []
rent_l = rent_df.sum(axis=1).tolist()

x= 0
rent_bracket = []
rent_tier = []
for value in rent_l:
    
    rent_med_num.append(value / 2)
    
rent_med_num

for index, row in rent_df.iterrows():
    count=0
    y=0
    for colname in rent_df.columns:
        
        if count <= rent_med_num[x]:
            
            count += row[colname]
            y+=1
            
        elif count > rent_med_num[x]:
            
            rent_bracket.append(colname)
            rent_tier.append(y)
            
            break
    x += 1

In [ ]:
rent_bracket = rent_df['Median Rent'].tolist()

In [ ]:
rent_endpoints = list(np.mgrid[min(rent_bracket):max(rent_bracket):12j])
#rent_fig = ff.create_choropleth()
mortgage_price_colorscale = ["#cccccc","#ebf3fb","#deebf7","#d2e3f3","#c6dbef","#b3d2e9","#9ecae1",
              "#85bcdb","#6baed6","#57a0ce","#4292c6","#3082be","#2171b5","#1361a9",
              "#08519c","#0b4083","#08306b"]
rent_endpoints

In [ ]:
rent_fig = ff.create_choropleth(fips = fips_list_with_state, values = rent_bracket,
                                        colorscale = mortgage_price_colorscale, binning_endpoints = rent_endpoints,
                                         scope = ['CA'],
                                         county_outline={'color': 'rgb(255,255,255)', 'width': .5}, legend_title = 'Amount of Rent',
                                    title = 'Median Rent by County')
py.iplot(rent_fig, filename='Housing Price by County')

In [ ]:
pio.write_image(rent_fig, 'images/Rent by County.png')

In [ ]:
edportance=int(input("How important is it that your neighbors are well educated?: "))
wealthportance=int(input("How important is it that your neighbors are wealthy?: "))
#comportance= int(input("How important is it that your commute is short? "))
#GET USER INPUT
print("Please rank the importance of the following factors. Note: 1 most important 2 least important")
print("")
#get user input about ranks
edrank= int(input("My neighbors are well educated: "))
wealthrank= int(input("My neighbors are rich: "))
#comrank= int(input("I live close to where I work: "))
print("")
#invert values to correlate value instead of rank with importance
edmult=4-edrank
wealthmult=4-wealthrank
#commult=4-comrank


In [ ]:
#get combined importance multipliers from user input section at top
edfinmult=edmult*edportance
wealthfinmult=wealthmult*wealthportance

#before making final DataFrame, we'll need to list cities and then locate each type of ranking record per city
#get countynames from a DataFrame that DOES NOT use it as an index
#countieslist=rent_per_df['County Name'].tolist() #be careful about executing the list this actually comes from
#numcounties=len(countieslist)
#finalcountyinfo=pd.DataFrame(countieslist)

#df.loc["Berry", "Phone Number"]

#final wealth rankings DataFrame construction
isolated_wealth_rankings=income_ranking_df.loc[:, ['Wealth Rank']]
isolated_wealth_rankings_mult=isolated_wealth_rankings
#lambda function to switch rank to score and multiply it by preferences
isolated_wealth_rankings_mult['Wealth Score']=isolated_wealth_rankings_mult["Wealth Rank"].apply(lambda x : (58-x)*wealthfinmult)
#isolated_wealth_rankings_mult['Wealth Rank'].apply(lambda x : x*wealthfinmult)
isolated_wealth_rankings_mult
#df['a'] = df['a'].apply(lambda x: x + 1)

isolated_education_rankings=education_ranking_df.loc[:, ['Education Rank']]
isolated_education_rankings_mult=isolated_education_rankings
isolated_education_rankings_mult['Education Score']=isolated_education_rankings_mult['Education Rank'].apply(lambda x: (58-x)*edfinmult)
#isolated_education_rankings_mult['Education Rank'].apply(lambda x: x*edfinmult)
#isolated_education_rankings
#isolated_education_rankings_mult

final_preferences_df=pd.merge(isolated_education_rankings_mult, isolated_wealth_rankings_mult, on='County Name', how='outer')
final_preferences_df['Final County Score']=final_preferences_df['Education Score']+final_preferences_df['Wealth Score']

final_rank_df=final_preferences_df.sort_values('Final County Score', ascending=False)
final_rank_df['Overall County Rank']=ranks
final_rank_df

#USE THIS METHOD TO MAKE FINAL DF? finalpreferences_df=pd.concat([df1['c'], df2['c']], axis=1, keys=['df1', 'df2'])